# This version tries to use almost all SQL (besides python wrapper)

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# define the directory that has the struct3dp data files
modelname='Run_230'
rundir='/Users/ulberg/research/MSH/crosson/Runs'
datadir='{}/{}/Data'.format(rundir,modelname)

# These file names are always associated with an inversion
stafile='combined.sta' # the stations that recorded travel times
eqobsfile='quakes.obs' # the travel time observations for earthquakes
eqlocfile='quakes.loc' # the locations of earthquakes
exobsfile='explos.obs' # the travel time observations for explosions
exlocfile='explos.loc' # the locations of explosions

# read into dataframes
sta=pd.read_csv(datadir + '/' + stafile, names=['Station','Longitude','Latitude','Depth'], sep=' ', skipinitialspace=True)
obs_eq=pd.read_csv(datadir + '/' + eqobsfile, names=['SourceID','Station','Phase','ArrivalTime','Uncertainty'], sep=' ', skipinitialspace=True)
loc_eq=pd.read_csv(datadir + '/' + eqlocfile, names=['SourceID','Longitude','Latitude','Depth','EventTime'], sep=' ', skipinitialspace=True)
obs_ex=pd.read_csv(datadir + '/' + exobsfile, names=['SourceID','Station','Phase','ArrivalTime','Uncertainty'], sep=' ', skipinitialspace=True)
loc_ex=pd.read_csv(datadir + '/' + exlocfile, names=['SourceID','Longitude','Latitude','Depth','EventTime'], sep=' ', skipinitialspace=True)

# read in more files that provide some mapping between other files
mapfile='map.map'
antUWfile='AntUWreview.txt'

mapcols=['dbname','orid','SourceID','method','Longitude-ant','Latitude-ant','Depth-ant','FullTime']
antUWcols=['PNSNid','dbname','orid','picker','Latitude-uw','Longitude-uw','Depth-uw']

datamap=pd.read_csv(datadir + '/' + mapfile, names=mapcols, sep=' ', skipinitialspace=True)
antUW=pd.read_csv('{}/{}/ANT/{}'.format(rundir,modelname,antUWfile), names=antUWcols, sep=' ', skipinitialspace=True, dtype={'orid': str})

In [4]:
print('datamap')
print(datamap.head())
print('antUW')
print(antUW.head())

datamap
  dbname orid  SourceID      method  Longitude-ant  Latitude-ant  Depth-ant  \
0  SHOTS  151  8000151p  shots2s3dp     -121.55266      46.71370     -0.668   
1  SHOTS  152  8000152p  shots2s3dp     -121.77356      46.20077     -1.029   
2  SHOTS  155  8000155p  shots2s3dp     -121.99266      46.18471     -1.009   
3  SHOTS  156  8000156p  shots2s3dp     -122.44470      46.04884     -0.834   
4  SHOTS  164  8000164p  shots2s3dp     -122.13281      46.08131     -0.594   

              FullTime  
0  20140724T113500.020  
1  20140725T050000.020  
2  20140725T072000.020  
3  20140725T073500.020  
4  20140725T085000.020  
antUW
     PNSNid   dbname  orid picker  Latitude-uw  Longitude-uw  Depth-uw
0  60810532  2014_Q3     2   Carl      46.5397     -122.9978       0.0
1  60810642  2014_Q3  1394   Carl      45.8663     -122.4492       0.0
2  60058633  2014_Q3  1395   Carl      45.5633     -122.6557      15.9
3  60814162  2014_Q3  1396   Carl      46.1728     -122.4225       0.0
4     

In [2]:
### add data to sqlite database

dbdir='/Users/ulberg/Documents/GitHub/travelTimeDB/DB' # make sure this directory is created before running
filename=dbdir + '/tt_db_sql2.sqlite'

# create engine
engine=create_engine('sqlite:///' + filename)

### create tables in 'raw' state (with '_r'), with all columns
### will work on these in sql to create the final tables
# add station table
sta.to_sql('sta_r', con=engine, if_exists='replace', index=False) # convert dataframe to sqlite db, replace old version (DANGEROUS)
obs_eq.to_sql('obs_r', con=engine, if_exists='replace', index=False)
loc_eq.to_sql('loc_r', con=engine, if_exists='replace', index=False)
datamap.to_sql('map_r', con=engine, if_exists='replace', index=False)
antUW.to_sql('antUW_r', con=engine, if_exists='replace', index=False)


# Now start manipulating tables to get final usable products
Do commands with 'engine.execute("sqlCommand")'  
example: engine.execute("SELECT * FROM station") or engine.execute("PRAGMA table_info('station')")  

Or use sqlite3 terminal command window

### Final tables wanted {tablename: columnnames}
{station: ['Station','Longitude','Latitude','Depth']}  
{observation: ['SourceID','Station','Phase','Uncertainty','FullArrivalTime','ArrivalTime']}  
{event: ['SourceID','Longitude','Latitude','Depth','PNSNid','Longitude-uw','Latitude-uw','Depth-uw','FullTime','EventTime']}

### Current tables {tablename:columnnames}
{sta_r: ['Station', 'Longitude', 'Latitude', 'Depth']} # already done with this one (becomes station table)!!!  
{obs_r: ['SourceID', 'Station', 'Phase', 'ArrivalTime', 'Uncertainty']} # need to add FullArrivalTime, then it will be observation table  
{loc_r: ['SourceID', 'Longitude', 'Latitude', 'Depth', 'EventTime']} # this will become event, need to add locations and id from antUW_r, and FullTime from map_r  
{map_r: ['dbname', 'orid', 'SourceID', 'method', 'Longitude-ant', 'Latitude-ant', 'Depth-ant', 'FullTime']}  
{antUW_r: ['PNSNid', 'dbname', 'orid', 'picker', 'Latitude-uw', 'Longitude-uw', 'Depth-uw']}

In [19]:
# check all the names of the tables
engine.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%'").fetchall()

[('sta_r',), ('obs_r',), ('loc_r',), ('map_r',), ('antUW_r',)]

In [34]:
# look at the first few rows of a given table
tablename='sta_r'
engine.execute("SELECT * FROM {} limit 10".format(tablename)).fetchall()


[('MB05', -122.28102, 46.620870000000004, -0.6409999999999999),
 ('MB07', -122.04239, 46.62378, -0.878),
 ('MC06', -122.1572, 46.55202, -0.77),
 ('MC08', -121.96321999999999, 46.55335, -0.653),
 ('MD05', -122.27816999999999, 46.52316, -0.718),
 ('MD09', -121.70671000000002, 46.50288, -0.9590000000000001),
 ('MD12', -121.22067, 46.47694, -1.496),
 ('ME02', -122.70733999999999, 46.43916, -0.158),
 ('ME03', -122.57647, 46.45207, -0.31),
 ('ME04', -122.35193999999998, 46.45061, -0.73)]

In [36]:
# check the column names of a given table
tablename='antUW_r'
out=engine.execute("PRAGMA table_info('{}')".format(tablename)).fetchall()
col=[x[1] for x in out]
col

['PNSNid',
 'dbname',
 'orid',
 'picker',
 'Latitude-uw',
 'Longitude-uw',
 'Depth-uw']

## Create final station table
> CREATE TABLE station AS  
-- SELECT * FROM sta_r;

# Start working on observation table
Need to add FullArrivalTime to obs_r, then it will be observation table
FullArrivalTime = map_r.FullTime + (obs_r.ArrivalTime - loc_r.EventTime) 
The indices for each of these don't necessarily match up, so need to use SourceID to match them, with left outer joins on the obs_r table

## Commands to use

### First attempts, looking at the data
Create new table by joining some others using SourceID - this will have the wrong number of rows since map_r has duplicates
> CREATE TABLE obs_r1 AS  
-- SELECT obs_r.\*, loc_r.EventTime, map_r.FullTime FROM obs_r  
-- LEFT OUTER JOIN map_r USING(SourceID)  
-- LEFT OUTER JOIN loc_r USING(SourceID);

Check length of table
> SELECT count(\*) FROM obs_r1;

Look at first few rows of table
> SELECT \* FROM obs_r1 limit 10;

Issue with multiple records per SourceID in map_r, so
First, show map_r rows with duplicate SourceID
> SELECT m.\* FROM map_r m INNER JOIN  
-- (SELECT SourceID, COUNT(\*) FROM map_r  
-- GROUP BY SourceID HAVING COUNT(\*) > 1 )  
-- USING(SourceID) ORDER BY m.SourceID, m.method ASC limit 10;

### Convert loc_r.FullTime to FullEventTime in correct sql time format
First add FullTime from map_r
> CREATE TABLE loc_r1 AS  
SELECT loc_r.\*, m.FullTime FROM loc_r  
LEFT OUTER JOIN ( SELECT DISTINCT SourceID, FullTime FROM map_r ORDER BY SourceID) m USING(SourceID);

Now convert FullTime to correct format
> CREATE TABLE loc_r2 AS  
SELECT SourceID,Longitude,Latitude,Depth,EventTime,substr(FullTime,1,4) || '-' || substr(FullTime,5,2) || '-' || substr(FullTime,7,5) || ':' || substr(FullTime,12,2) || ':' || substr(FullTime,14) AS FullEventTime FROM loc_r1;

### Add stuff to new observation table
#### Previous versions that needed map_r because  loc_r hadn't been updated yet
This will just use one match of SourceID from map_r (possible because the FullTime will also be a duplicate)
> CREATE TABLE obs_r1 AS  
-- SELECT obs_r.\*, m.FullTime FROM obs_r  
-- LEFT OUTER JOIN ( SELECT DISTINCT SourceID, FullTime FROM map_r ORDER BY SourceID) m USING(SourceID);

Add in the loc_r info to the previous command
> CREATE TABLE obs_r1 AS  
-- SELECT obs_r.\*, m.FullTime, loc_r.EventTime FROM obs_r  
-- LEFT OUTER JOIN ( SELECT DISTINCT SourceID, FullTime FROM map_r ORDER BY SourceID) m USING(SourceID)  
-- LEFT OUTER JOIN loc_r USING(SourceID);

#### New version since loc_r has been updated to loc_r2
> CREATE TABLE obs_r1 AS  
SELECT obs_r.\*, loc_r2.EventTime, loc_r2.FullEventTime FROM obs_r  
LEFT OUTER JOIN loc_r2 USING(SourceID);

### Now need to make FullArrivalTime=FullTime + ArrivalTime - EventTime  
> CREATE TABLE obs_r2 AS  
-- SELECT SourceID,Station,Phase,ArrivalTime,Uncertainty,EventTime,FullEventTime,strftime('%Y-%m-%dT%H:%M:%f',FullEventTime,'-'||EventTime||' seconds','+'||ArrivalTime||' seconds') AS FullArrivalTime FROM obs_r1;

## Make final observation table
> CREATE TABLE observation AS  
-- SELECT SourceID,Station,Phase,Uncertainty,FullArrivalTime FROM obs_r2;

# Do event table
To get:  
{event: ['SourceID','Longitude','Latitude','Depth','PNSNid','Longitude-uw','Latitude-uw','Depth-uw','FullTime','EventTime']}  

To start, already have:
{loc_r: ['SourceID', 'Longitude', 'Latitude', 'Depth', 'EventTime']}

Need fields from:
{antUW_r: ['PNSNid', 'dbname', 'orid', 'picker', 'Latitude-uw', 'Longitude-uw', 'Depth-uw']}  
{map_r: ['dbname', 'orid', 'SourceID', 'method', 'Longitude-ant', 'Latitude-ant', 'Depth-ant', 'FullTime']}

Do outer join using SourceID for loc_r, map_r to get FullTime (actually already done above)  
To get PNSNid from ant_UW_r, need to construct the SourceID from dbname and orid, then do outer join with loc_r (also to get locations)

## Commands to use
### First need to convert dbname and orid to SourceID in antUW_r
This will work correctly for the Antelope sources, but not for the PNSN sources
> SELECT substr(dbname,4,1)||substr(dbname,7,1)||'0'||printf("%04d",orid)||'p' AS id1 FROM antUW_r;

Could try joining from the map, first into antUW_r
> CREATE TABLE antUW_r1 AS  
-- SELECT a.\*, m.SourceID FROM antUW_r a  
-- LEFT OUTER JOIN map_r m USING(dbname,orid);

### Now join the antUW information with loc_r2
> CREATE TABLE loc_r3 AS  
-- SELECT l.\*, a.PNSNid, a."Longitude-uw", a."Latitude-uw", a."Depth-uw" FROM loc_r2 l  
-- LEFT OUTER JOIN antUW_r1 a USING(SourceID);

## Make final event location table
> CREATE TABLE event AS  
-- SELECT SourceID,PNSNid,Longitude,Latitude,Depth,FullEventTime,"Longitude-uw","Latitude-uw","Depth-uw" FROM loc_r3;